In [1]:
import json
import pandas as pd

In [2]:
import os
from gensim import matutils
from pyvi import ViTokenizer
import string
import re
import numpy as np
import regex as re
import math

class Char:
    UNICHARS = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
    UNSIGNCHARS = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"
    CHAR1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
    CHARUTF8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ"
    VOWEL_TABLE = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                   ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                   ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                   ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                   ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                   ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                   ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                   ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                   ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                   ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                   ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                   ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]

    FIRST_CHAR_TABLE = ['', 'f', 's', 'r', 'x', 'j']
    REGEX_UNICODE_TEMPLATE = r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
    EMOJI_PATTERN = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)

uniChars = Char.UNICHARS
unsignChars = Char.UNSIGNCHARS
PUNCT_TO_REMOVE = string.punctuation

vowel_table = Char.VOWEL_TABLE
first_char_table = Char.FIRST_CHAR_TABLE
vowel_to_ids = {}


def loaddicchar():
    dic = {}
    char1252 = Char.CHAR1252.split(
        '|')
    charutf8 = Char.CHARUTF8.split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic


dicchar = loaddicchar()


def gen_text(object):
    res = []
    if object['detailed_address'] == object['detailed_address']:
        text = f"Bán nhà tại {object['detailed_address']}"
        res.append(text)
    if math.isnan(object['number_of_floors']):
        res.append(f"Xây {object['number_of_floors']} tầng")
        res.append(f"{object['number_of_floors']}t")
        res.append(f"Thiết kế xây {object['number_of_floors']}t")
        res.append(f"Thiết kế xây {object['number_of_floors']} tầng")

    if math.isnan(object['acreage']):
        res.append(f"Diện tích {object['acreage']}m²")
        res.append(f"{object['acreage']}m²")
        if object['detailed_address'] == object['detailed_address']:
            res.append(
                f"Bán nhà tại {object['detailed_address']}, {object['acreage']}m")

    if math.isnan(object['facede']):
        res.append(f"{object['facede']}M MT")

    if math.isnan(object['number_of_bedrooms']):
        res.append(f"Nhà có {object['facede']} phòng ngủ")
        res.append(f"{object['facede']} phòng ngủ")

    if math.isnan(object['is_car_road']):
        if object['is_car_road'] == 0:
            res.append(
                f"Nhà cách ngõ ô tô tránh {object['cach_ngo_o_to_tranh']}m")
            res.append(
                f"Cách {object['cach_ngo_o_to_tranh']}m ra ngõ ô tô tránh")
            res.append(
                f"Cách Mặt Ngõ Ô Tô Tránh {object['cach_ngo_o_to_tranh']}m")
        elif object['is_car_road'] == 1:
            res.append(f"Nhà ở ngay mặt ngõ")
            res.append(f"Nhà ở mặt ngõ")

    res = '. '.join(res)
    res = res.strip()

    return res


for i in range(len(vowel_table)):
    for j in range(len(vowel_table[i]) - 1):
        vowel_to_ids[vowel_table[i][j]] = (i, j)


def standardize_vietnamese_word_sign(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    sentence_sign = 0
    vowel_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = vowel_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            sentence_sign = y
            chars[index] = vowel_table[x][0]
        if not qu_or_gi or index != 1:
            vowel_index.append(index)
    if len(vowel_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = vowel_to_ids.get(chars[1])
                chars[1] = vowel_table[x][sentence_sign]
            else:
                x, y = vowel_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = vowel_table[x][sentence_sign]
                else:
                    chars[1] = vowel_table[5][sentence_sign] if chars[1] == 'i' else vowel_table[9][sentence_sign]
            return ''.join(chars)
        return word

    for index in vowel_index:
        x, y = vowel_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = vowel_table[x][sentence_sign]
            # for index2 in vowel_index:
            #     if index2 != index:
            #         x, y = vowel_to_ids[chars[index]]
            #         chars[index2] = vowel_table[x][0]
            return ''.join(chars)

    if len(vowel_index) == 2:
        if vowel_index[-1] == len(chars) - 1:
            x, y = vowel_to_ids[chars[vowel_index[0]]]
            chars[vowel_index[0]] = vowel_table[x][sentence_sign]
            # x, y = vowel_to_ids[chars[vowel_index[1]]]
            # chars[vowel_index[1]] = vowel_table[x][0]
        else:
            # x, y = vowel_to_ids[chars[vowel_index[0]]]
            # chars[vowel_index[0]] = vowel_table[x][0]
            x, y = vowel_to_ids[chars[vowel_index[1]]]
            chars[vowel_index[1]] = vowel_table[x][sentence_sign]
    else:
        # x, y = vowel_to_ids[chars[vowel_index[0]]]
        # chars[vowel_index[0]] = vowel_table[x][0]
        x, y = vowel_to_ids[chars[vowel_index[1]]]
        chars[vowel_index[1]] = vowel_table[x][sentence_sign]
        # x, y = vowel_to_ids[chars[vowel_index[2]]]
        # chars[vowel_index[2]] = vowel_table[x][0]
    return ''.join(chars)


def is_valid_vietnam_word(word):
    chars = list(word)
    vowel_index = -1
    for index, char in enumerate(chars):
        x, y = vowel_to_ids.get(char, (-1, -1))
        if x != -1:
            if vowel_index == -1:
                vowel_index = index
            else:
                if index - vowel_index != 1:
                    return False
                vowel_index = index
    return True


def standardize_vietnamese_sentence_sign(sentence):

    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)',
                    r'\1/\2/\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = standardize_vietnamese_word_sign(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)


def covert_unicode(txt):
    return re.sub(
        Char.REGEX_UNICODE_TEMPLATE,
        lambda x: dicchar[x.group()], txt)


def preprocess_text(text):
    try:
        text = text.lower()
        text = covert_unicode(text)
        text = standardize_vietnamese_sentence_sign(text)
        return text
    except:
        return np.nan


def remove_emoji(text):
    emoji_pattern = Char.EMOJI_PATTERN
    return emoji_pattern.sub(r'', text)


def remove_special_character(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
    text = text.replace('\n', '')
    text = text.strip()
    words = text.split(" ")
    words = [word for word in words if word != ""]
    return " ".join(words)




In [3]:
data = json.load(open('../data/process_v1/process_data.json', 'r'))
data[0]

{'numberOfFloors': 1,
 'ward': 'Văn Quán',
 'street': 'Chiến Thắng',
 'district': 'Hà Đông',
 'city': 'Hà Nội',
 'lat': 20.978546,
 'lon': 105.796775,
 'typeOfRealEstate': 'privateProperty',
 'frontWidth': 0,
 'endWidth': 0,
 'facade': 'oneSideOpen',
 'houseDirection': None,
 'landSize': 48,
 'price': 4.35,
 'unitPrice': 'billion',
 'distanceToNearestRoad': 0,
 'frontRoadWidth': 0,
 'accessibility': 'fitTwoCars',
 'landType': 'residentialLand',
 'description': '<div>-\tBán gấp nhà đường Chiến Thắng, đầu quận Hà Đông, dễ dàng vào trung tâm HN, sang quận Thanh Xuân, Nam Từ Liêm, gần các trường ĐH An Ninh, Bưu chính…</div><div>- Diện tích 48m2, phân lô vuông vức, ô tô đỗ tận cửa, khu vực dân trí cao, văn minh, nhà có thể ở ngay.</div><div>-\tSổ đỏ sạch đẹp sẵn sàng giao dịch.</div><div>-\tCam kết: không thu bất kể loại phí gì kể cả khi giao dịch thành công.</div><div>-\tGiá: 4,35 tỷ (Có thương lượng với khách thiện chí).</div><div>-\tMiễn tiếp trung gian và môi giới.</div><div>-\tLiên hệ 

In [4]:
data = [item for item in data if item['street'] != None]

In [5]:
len(data)

236192

In [6]:
data = [item for item in data if item['district'] != None]

In [7]:
len(data)

236192

In [8]:
data = [item for item in data if item['lat'] != None and item['lon'] != None]

In [9]:
data = [item for item in data if item['landSize'] != None and item['price'] != None]

In [10]:
len(data)

236192

In [11]:
data = pd.DataFrame(data)

In [30]:
data

,numberOfFloors,ward,street,district,city,lat,lon,typeOfRealEstate,frontWidth,endWidth,facade,houseDirection,landSize,price,unitPrice,distanceToNearestRoad,frontRoadWidth,accessibility,landType,description
0,1,Văn Quán,Chiến Thắng,hà đông,hà nội,20.978546,105.796775,privateProperty,0.0,0.0,oneSideOpen,None,48.0,4.35,billion,0,0,fitTwoCars,residentialLand,"<div>-\tBán gấp nhà đường Chiến Thắng, đầu quậ..."
1,1,Văn Quán,Chiến Thắng,hà đông,hà nội,20.978546,105.796775,privateLand,0.0,0.0,oneSideOpen,None,40.0,4.68,billion,0,0,fitOneCarAndOneMotorbike,residentialLand,Đang làm ăn lớn bán đất vị trí thuận lợi tọa l...
2,5,Văn Quán,Trần Phú,hà đông,hà nội,20.975468,105.781978,privateProperty,4.0,4.0,oneSideOpen,None,40.0,5.59,billion,0,0,notInTheAlley,residentialLand,"Bán nhà vào ở ngay giá mềm chỉ 5.59 tỷ, có diệ..."
3,5,Phúc La,19/5,hà đông,hà nội,20.971822,105.783025,semiDetachedVilla,0.0,0.0,twoSideOpen,None,35.0,5.15,billion,0,0,notInTheAlley,residentialLand,"Thất nghiệp bán biệt thự, giá bán cực mềm 5.15..."
4,5,Văn Quán,Đại An,hà đông,hà nội,20.979634,105.790935,privateProperty,0.0,0.0,oneSideOpen,None,55.0,4.70,billion,0,0,fitOneCarAndOneMotorbike,residentialLand,"Bán nhà giá siêu khủng chỉ 4.7 tỷ,diện tích kh..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236187,1,,,nam từ liêm,hà nội,21.031929,105.766922,condominium,0.0,0.0,oneSideOpen,None,105.0,0.00,billion,0,0,notInTheAlley,residentialLand,Chủ nhà thiện chí cần bán gấp căn 105m² tại tò...
236188,1,,,long biên,hà nội,21.019056,105.906403,privateLand,7.0,0.0,twoSideOpen,None,75.0,19.00,billion,0,0,fitThreeCars,residentialLand,"Bán đất hiếm! Đất mặt phố Cổ Linh, vị trí gần ..."
236189,1,,,hà đông,hà nội,20.982687,105.781616,privateLand,5.0,0.0,oneSideOpen,None,32.0,4.95,billion,0,0,notInTheAlley,residentialLand,"Vị trí đất siêu đắc địa, ngõ thông rộng 6m kin..."
236190,1,,,long biên,hà nội,21.040499,105.904793,condominium,0.0,0.0,oneSideOpen,None,70.0,2.00,billion,0,0,notInTheAlley,residentialLand,Chính chủ gửi bán gấp căn hộ 70m² 2 ngủ 2 vs n...


In [31]:
data['district'] = data['district'].apply(preprocess_text)

/tmp/ipykernel_493843/2073545792.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['district'] = data['district'].apply(preprocess_text)


In [32]:
data['city'] = data['city'].apply(preprocess_text)

/tmp/ipykernel_493843/1701522976.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['city'] = data['city'].apply(preprocess_text)


In [33]:
data = data[data['city'].isin(['hà nội', 'hồ chí minh'])]

In [34]:
len(data)

209713

In [35]:
data[data['city'] == 'hà nội']['district'].value_counts()

district
hà đông         19964
nam từ liêm     16336
cầu giấy        13813
đống đa         13640
long biên       12326
thanh xuân      10181
hoàng mai        9783
tây hồ           7988
hai bà trưng     7418
gia lâm          7144
bắc từ liêm      5657
ba đình          5645
hoài đức         4946
thanh trì        3394
đông anh         2533
hoàn kiếm        2221
mê linh           733
thạch thất        395
thường tín        346
sơn tây           327
ba vì             309
sóc sơn           212
đan phượng        195
thanh oai         166
quốc oai          143
chương mỹ          81
phúc thọ           51
ứng hòa            10
phú xuyên           6
mỹ đức              2
Name: count, dtype: int64

In [36]:
hn_df = data[data['city'] == 'hà nội']
hn_df['district'] = hn_df['district'].replace('mê linh', 'suburb_west')
hn_df['district'] = hn_df['district'].replace('ba vì', 'suburb_west')
hn_df['district'] = hn_df['district'].replace('phúc thọ', 'suburb_west')
hn_df['district'] = hn_df['district'].replace('thạch thất', 'suburb_west')
hn_df['district'] = hn_df['district'].replace('mỹ đức', 'suburb_west')
hn_df['district'] = hn_df['district'].replace('sơn tây', 'suburb_west')
hn_df['district'] = hn_df['district'].replace('quốc oai', 'suburb_west')
hn_df['district'] = hn_df['district'].replace('quốc oai', 'suburb_west')

hn_df['district'] = hn_df['district'].replace('sóc sơn', 'suburb_north')
hn_df['district'] = hn_df['district'].replace('đan phượng', 'suburb_north')

hn_df['district'] = hn_df['district'].replace('thanh oai', 'suburb_south')
hn_df['district'] = hn_df['district'].replace('ứng hòa', 'suburb_south')
hn_df['district'] = hn_df['district'].replace('phú xuyên', 'suburb_south')
hn_df['district'] = hn_df['district'].replace('thường tín', 'suburb_south')
hn_df['district'] = hn_df['district'].replace('chương mỹ', 'suburb_south')

/tmp/ipykernel_493843/2787100113.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hn_df['district'] = hn_df['district'].replace('mê linh', 'suburb_west')
/tmp/ipykernel_493843/2787100113.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hn_df['district'] = hn_df['district'].replace('ba vì', 'suburb_west')
/tmp/ipykernel_493843/2787100113.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [37]:
hn_df['district'].value_counts()

district
hà đông         19964
nam từ liêm     16336
cầu giấy        13813
đống đa         13640
long biên       12326
thanh xuân      10181
hoàng mai        9783
tây hồ           7988
hai bà trưng     7418
gia lâm          7144
bắc từ liêm      5657
ba đình          5645
hoài đức         4946
thanh trì        3394
đông anh         2533
hoàn kiếm        2221
suburb_west      1960
suburb_south      609
suburb_north      407
Name: count, dtype: int64

In [38]:
hcm_df = data[data['city'] == 'hồ chí minh']

In [39]:
hcm_df

,numberOfFloors,ward,street,district,city,lat,lon,typeOfRealEstate,frontWidth,endWidth,facade,houseDirection,landSize,price,unitPrice,distanceToNearestRoad,frontRoadWidth,accessibility,landType,description
47852,1,Phú Thuận,Đào Trí,quận 7,hồ chí minh,10.731108,106.743275,condominium,0.0,0.0,oneSideOpen,south,73.0,2.3,billion,0,0,notInTheAlley,residentialLand,Bảng giá cập nhập mới nhất tại dự án Q7 Sài Gò...
48758,1,,,quận 7,hồ chí minh,10.722662,106.713181,condominium,0.0,0.0,oneSideOpen,south,140.0,7.7,billion,0,0,notInTheAlley,residentialLand,Kẹt tiền bán gấp căn hộ Garden Plaza 1 Phú Mỹ ...
55451,1,Phường 2,Hồng Hà,tân bình,hồ chí minh,10.813342,106.667146,otherTypesOfProperty,0.0,0.0,oneSideOpen,east,75.0,0.0,billion,0,0,notInTheAlley,residentialLand,Shyn Home&Decor cần cho thuê gấp căn hộ cao cấ...
57361,1,Phường 2,Phổ Quang,tân bình,hồ chí minh,10.805912,106.668067,otherTypesOfProperty,0.0,0.0,oneSideOpen,east,70.0,0.0,billion,0,0,notInTheAlley,residentialLand,Shyn Home&Decor cần cho thuê gấp căn hộ cao cấ...
62178,1,Phường 13,Lê Văn Sỹ,quận 3,hồ chí minh,10.786903,106.679468,privateProperty,12.0,0.0,oneSideOpen,None,239.0,42.5,billion,0,0,notInTheAlley,residentialLand,"Cần bán gấp nhà HXH 351 đường Lê Văn Sỹ, P13, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236178,1,,,quận 3,hồ chí minh,10.778017,106.689781,condominium,0.0,0.0,oneSideOpen,None,75.0,12.0,billion,0,0,notInTheAlley,residentialLand,Căn hộ: Hướng nhà: Đông - Nam.\n+ Số phòng ngủ...
236180,3,Phường 6,Hoàng Hoa Thám,bình thạnh,hồ chí minh,10.808067,106.688528,townhouse,0.0,0.0,twoSideOpen,None,178.0,22.0,billion,0,0,notInTheAlley,residentialLand,Mặt tiền ngay chợ cây Quéo - gần 180m - ngang ...
236183,5,Bến Thành,Sương Nguyệt Anh,quận 1,hồ chí minh,10.771924,106.689094,townhouse,0.0,0.0,twoSideOpen,None,125.0,85.0,billion,0,0,notInTheAlley,residentialLand,- Bán gấp nhà siêu khan hiếm mặt tiền Sương Ng...
236184,1,,,quận 4,hồ chí minh,10.766646,106.703476,condominium,0.0,0.0,oneSideOpen,None,50.0,3.3,billion,0,0,notInTheAlley,residentialLand,"Diện tích: 50 m² (1 PN, 1 WC).\nTình trạng: Fu..."


In [45]:
df = pd.concat([hn_df, hcm_df])

In [46]:
df = df.sample(frac = 1.0)

In [47]:
df

,numberOfFloors,ward,street,district,city,lat,lon,typeOfRealEstate,frontWidth,endWidth,facade,houseDirection,landSize,price,unitPrice,distanceToNearestRoad,frontRoadWidth,accessibility,landType,description
120937,1,Phước Vĩnh An,Nguyễn Thị Lăng,củ chi,hồ chí minh,10.969329,106.533562,privateLand,5.0,0.0,oneSideOpen,None,82.6,0.75,billion,0,0,fitTwoCars,residentialLand,Ngộp cần bán gấp lô đất 85m đường Nguyễn Thị L...
108836,4,Tân Mai,Tân Mai,hoàng mai,hà nội,20.983778,105.849492,privateProperty,4.5,0.0,oneSideOpen,None,30.0,3.75,billion,0,0,parkCar,residentialLand,"HIẾM! CC BÁN GẤP NHÀ ngõ 258 TÂN MAI, Q.HOÀNG ..."
124022,1,Phường 1,Lạc Long Quân,quận 11,hồ chí minh,10.764317,106.642181,privateProperty,0.0,0.0,oneSideOpen,None,90.0,8.50,billion,0,0,notInTheAlley,residentialLand,"Tổng 90m² chia ra chỉ 94tr/m² đường 8m xe tải,..."
194658,3,Tân Quy,Mai Văn Vĩnh,quận 7,hồ chí minh,10.740684,106.713715,privateProperty,8.5,0.0,oneSideOpen,None,85.0,0.00,billion,0,0,notInTheAlley,residentialLand,"Chủ gửi bán nhanh nhà đẹp, 3 tầng, xây dựng ki..."
64812,5,Yên Hòa,Nguyễn Chánh,cầu giấy,hà nội,21.018970,105.786560,townhouse,26.0,0.0,twoSideOpen,None,190.0,92.00,billion,0,0,notInTheAlley,residentialLand,Chính chủ gửi bán shophouse lô góc mặt phố Ngu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101141,1,Thụy Khuê,Thụy Khuê,tây hồ,hà nội,21.042545,105.824885,condominium,0.0,0.0,oneSideOpen,None,130.0,10.00,billion,0,0,notInTheAlley,residentialLand,Không còn nhu cầu ở nên tôi bán cắt lỗ căn hộ ...
256,4,Văn Quán,Văn Quán,hà đông,hà nội,20.978724,105.796025,privateProperty,0.0,0.0,oneSideOpen,None,50.0,4.96,billion,0,0,theBottleNeckPoint,residentialLand,Bán nhà ở có diện tích chính 50m2 ngay Hà Đông...
106999,3,Sơn Kỳ,Lê Trọng Tấn,tân phú,hồ chí minh,10.810419,106.612655,privateProperty,4.0,0.0,oneSideOpen,None,61.0,0.00,billion,0,0,parkCar,residentialLand,LH Vương \n0767 452 758\n KDTL bán gấp nhà đườ...
1680,5,Phúc La,19/5,hà đông,hà nội,20.971822,105.783025,privateProperty,0.0,0.0,oneSideOpen,None,40.0,4.50,billion,0,0,theBottleNeckPoint,residentialLand,"<p>Bán nhà đường 19/5 Văn Quán, nhà đẹp, gần p..."


In [48]:
df['ward'] = df['ward'].apply(preprocess_text)
df['street'] = df['street'].apply(preprocess_text)
df['description'] = df['description'].apply(preprocess_text)

In [49]:
df

,numberOfFloors,ward,street,district,city,lat,lon,typeOfRealEstate,frontWidth,endWidth,facade,houseDirection,landSize,price,unitPrice,distanceToNearestRoad,frontRoadWidth,accessibility,landType,description
120937,1,phước vĩnh an,nguyễn thị lăng,củ chi,hồ chí minh,10.969329,106.533562,privateLand,5.0,0.0,oneSideOpen,None,82.6,0.75,billion,0,0,fitTwoCars,residentialLand,ngộp cần bán gấp lô đất 85m đường nguyễn thị l...
108836,4,tân mai,tân mai,hoàng mai,hà nội,20.983778,105.849492,privateProperty,4.5,0.0,oneSideOpen,None,30.0,3.75,billion,0,0,parkCar,residentialLand,"hiếm! cc bán gấp nhà ngõ 258 tân mai, q.hoàng ..."
124022,1,phường 1,lạc long quân,quận 11,hồ chí minh,10.764317,106.642181,privateProperty,0.0,0.0,oneSideOpen,None,90.0,8.50,billion,0,0,notInTheAlley,residentialLand,"tổng 90m² chia ra chỉ 94trm² đường 8m xe tải, ..."
194658,3,tân quy,mai văn vĩnh,quận 7,hồ chí minh,10.740684,106.713715,privateProperty,8.5,0.0,oneSideOpen,None,85.0,0.00,billion,0,0,notInTheAlley,residentialLand,"chủ gửi bán nhanh nhà đẹp, 3 tầng, xây dựng ki..."
64812,5,yên hòa,nguyễn chánh,cầu giấy,hà nội,21.018970,105.786560,townhouse,26.0,0.0,twoSideOpen,None,190.0,92.00,billion,0,0,notInTheAlley,residentialLand,chính chủ gửi bán shophouse lô góc mặt phố ngu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101141,1,thụy khuê,thụy khuê,tây hồ,hà nội,21.042545,105.824885,condominium,0.0,0.0,oneSideOpen,None,130.0,10.00,billion,0,0,notInTheAlley,residentialLand,không còn nhu cầu ở nên tôi bán cắt lỗ căn hộ ...
256,4,văn quán,văn quán,hà đông,hà nội,20.978724,105.796025,privateProperty,0.0,0.0,oneSideOpen,None,50.0,4.96,billion,0,0,theBottleNeckPoint,residentialLand,bán nhà ở có diện tích chính 50m2 ngay hà đông...
106999,3,sơn kỳ,lê trọng tấn,tân phú,hồ chí minh,10.810419,106.612655,privateProperty,4.0,0.0,oneSideOpen,None,61.0,0.00,billion,0,0,parkCar,residentialLand,lh vương 0767 452 758 kdtl bán gấp nhà đường l...
1680,5,phúc la,195,hà đông,hà nội,20.971822,105.783025,privateProperty,0.0,0.0,oneSideOpen,None,40.0,4.50,billion,0,0,theBottleNeckPoint,residentialLand,"<p>bán nhà đường 195 văn quán, nhà đẹp, gần ph..."


In [50]:
df

,numberOfFloors,ward,street,district,city,lat,lon,typeOfRealEstate,frontWidth,endWidth,facade,houseDirection,landSize,price,unitPrice,distanceToNearestRoad,frontRoadWidth,accessibility,landType,description
120937,1,phước vĩnh an,nguyễn thị lăng,củ chi,hồ chí minh,10.969329,106.533562,privateLand,5.0,0.0,oneSideOpen,None,82.6,0.75,billion,0,0,fitTwoCars,residentialLand,ngộp cần bán gấp lô đất 85m đường nguyễn thị l...
108836,4,tân mai,tân mai,hoàng mai,hà nội,20.983778,105.849492,privateProperty,4.5,0.0,oneSideOpen,None,30.0,3.75,billion,0,0,parkCar,residentialLand,"hiếm! cc bán gấp nhà ngõ 258 tân mai, q.hoàng ..."
124022,1,phường 1,lạc long quân,quận 11,hồ chí minh,10.764317,106.642181,privateProperty,0.0,0.0,oneSideOpen,None,90.0,8.50,billion,0,0,notInTheAlley,residentialLand,"tổng 90m² chia ra chỉ 94trm² đường 8m xe tải, ..."
194658,3,tân quy,mai văn vĩnh,quận 7,hồ chí minh,10.740684,106.713715,privateProperty,8.5,0.0,oneSideOpen,None,85.0,0.00,billion,0,0,notInTheAlley,residentialLand,"chủ gửi bán nhanh nhà đẹp, 3 tầng, xây dựng ki..."
64812,5,yên hòa,nguyễn chánh,cầu giấy,hà nội,21.018970,105.786560,townhouse,26.0,0.0,twoSideOpen,None,190.0,92.00,billion,0,0,notInTheAlley,residentialLand,chính chủ gửi bán shophouse lô góc mặt phố ngu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101141,1,thụy khuê,thụy khuê,tây hồ,hà nội,21.042545,105.824885,condominium,0.0,0.0,oneSideOpen,None,130.0,10.00,billion,0,0,notInTheAlley,residentialLand,không còn nhu cầu ở nên tôi bán cắt lỗ căn hộ ...
256,4,văn quán,văn quán,hà đông,hà nội,20.978724,105.796025,privateProperty,0.0,0.0,oneSideOpen,None,50.0,4.96,billion,0,0,theBottleNeckPoint,residentialLand,bán nhà ở có diện tích chính 50m2 ngay hà đông...
106999,3,sơn kỳ,lê trọng tấn,tân phú,hồ chí minh,10.810419,106.612655,privateProperty,4.0,0.0,oneSideOpen,None,61.0,0.00,billion,0,0,parkCar,residentialLand,lh vương 0767 452 758 kdtl bán gấp nhà đường l...
1680,5,phúc la,195,hà đông,hà nội,20.971822,105.783025,privateProperty,0.0,0.0,oneSideOpen,None,40.0,4.50,billion,0,0,theBottleNeckPoint,residentialLand,"<p>bán nhà đường 195 văn quán, nhà đẹp, gần ph..."


In [52]:
df = df.reset_index(drop = True)
df.to_csv('../data/process_v1/process_data_2.csv', index = False)